# 工具链

cython默认提供了3个命令行工具

+ `cython`,用于将cython语言的源码转译为C代码
+ `cythonize`,用于将cython语言的源码编译为python可以加载的动态连接库.
+ `cygdb`,用于调试cython编译出来动态链接库的工具,提供了断点设置,变量查看,单步执行,堆栈追踪,查看内存这几个基本的调试功能


官方还提供了3个额外的工具

+ `cython_freeze`,用于创建一个C的入口函数并将指定的cython代码嵌入其中.
+ `cythonrun`,等效于`python -m Cython.Build.BuildExecutable [ARGS] somefile.py`,编译一个python脚本为一个可执行程序
+ `pcython`,命令行执行一段python代码或cython代码的,相当于python中的`python -c`

只是这三个工具都没有正式发布,其中`cythonrun`和`pcython`都已经好几年没有维护了,不建议使用.直接安装cython并不会被安装上面三个工具,如果需要可以去[项目下的bin文件夹](https://github.com/cython/cython/tree/master/bin)中复制对应的文件下来使用.

官方还提供了jupyter的魔法命令`cython`,使用时先试用`%load_ext cython`加载cython扩展,之后就可以使用了.

编辑器支持方面cython是比较弱的.vscode上的插件基本只有语法高亮功能.

我们使用一个简单的斐波那契数列计算作为例子,依次介绍这些工具.


In [35]:
%%writefile fib.pyx
# distutils: language=c++

def fib(int n):
    """Print the Fibonacci series up to n."""
    cdef int a = 0
    cdef int b = 1
    while b < n:
        print(b, end=' ')
        a, b = b, a + b
    print()

Overwriting fib.pyx


## cython

用于将cython语言的源码转译为C代码,它的主要参数可以有:

+ `-I`,指定include文件(`pxi`)所在文件夹,基本无用
+ `-o`,指定输出的c源文件
+ `-v`,转译过程可见
+ `-p`,将函数定义作为docstring
+ `-w`,指定工作目录,也就是cython模块所在的文件夹
+ `-a`,顺便生成一份html文件用于分析代码
+ `--cplus`,生成c++代码而非C代码
+ `-2`/`-3`/`-3str`,针对的python语法版本
+ `-X NAME=VALUE,...,`设置全局cython的转译指令
+ `-E NAME=VALUE,...,`设置全局转译时环境变量

In [36]:
!cython -a -3 fib.pyx

/Users/mac/micromamba/envs/py3.10/lib/python3.10/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPythonAsGlueLanguage/附录/Cython使用简介/工具链/fib.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


执行后会输出一个名为`helloworld.c`的c代码.使用`-a`可以额外输出一个html文件,其中黄色的部分就是与python交互的部分,也就是性能瓶颈.

## cythonize

cythonize是cython官方提供的编译工具,用起来和gcc差不太多.但是无法指定使用的C编译器,我们可以使用环境变量`CC`和`CXX`配置好编译器再执行.

常用的参数有:

+ `-i` 指明编译时是inplace模式,这种模式下编译好的动态链接库会放在源文件的同级目录中
+ `-a` 输出编译为C后各行对应的代码
+ `--3str`指定使用Python3的语法
+ `-X NAME=VALUE,...,` 设置cython转译器指令
+ `-E NAME=VALUE,...,` 设置转译时环境变量 
+ `-s NAME=VALUE,...,`,设置cython的选项
+ `-j n`,设置并行编译的并行度,默认1

具体的用法我们会在后文Cython的编译部分详细介绍.
  

In [38]:
!cythonize -i --3 fib.pyx

Compiling /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPythonAsGlueLanguage/附录/Cython使用简介/工具链/fib.pyx because it changed.
[1/1] Cythonizing /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPythonAsGlueLanguage/附录/Cython使用简介/工具链/fib.pyx


In [40]:
import fib

fib.fib(3)

1 1 2 


## cygdb

...施工中

## cython_freeze

cython_freeze的作用是给cython代码构造C语言的入口文件.它有两个应用

1. 纯用Cython写出可执行程序(当然还是必须带个python解释器的动态链接库)

2. 自定义python解释器,在其中嵌入cython写好的方法或类型

和python中一致,我们可以在cython中用`if __name__ == "__main__":`标识入口,这样`cython_freeze`会将第一个源码文件的`if __name__ == "__main__":`部分作为入口函数处理.

我们从纯用Cython写出可执行程序开始,用官方的例子来演示如何使用`cython_freeze`

+ `freeze/lcmath.pyx`

```cython
# cython: language_level=3

cdef extern from "math.h":
    double c_lgamma "lgamma" (double)
    double c_exp "exp" (double)


def exp(n):
    """Return e**n."""
    return c_exp(n)


def lfactorial(n):
    """Return an estimate of the log factorial of n."""
    return c_lgamma(n+1)


def factorial(n):
    """Return an estimate of the factorial of n."""
    return c_exp( c_lgamma(n+1) )


if __name__ == "__main__":
    import sys
    if len(sys.argv) != 2:
        sys.stderr.write("USAGE: %s n\nPrints n!.\n" % sys.argv[0])
        sys.exit(2)
    n, = map(float, sys.argv[1:])
    print(factorial(n))
```

+ `freeze/combinatorics.pyx`

```cython
# cython: language_level=3

import lcmath


def nCr(n, r):
    """Return the number of ways to choose r elements of a set of n."""
    return lcmath.exp(
        lcmath.lfactorial(n) -
        lcmath.lfactorial(r) -
        lcmath.lfactorial(n-r)
    )

if __name__ == "__main__":
    import sys
    if len(sys.argv) != 3:
        sys.stderr.write("USAGE: %s n r\nPrints n-choose-r.\n" % sys.argv[0])
        sys.exit(2)
    n, r = map(float, sys.argv[1:])
    print(nCr(n, r))
```

### 编译步骤

1. 转译入口c源码,这一步需要注意第一个文件的`__main__`部分会被转译为入口函数

In [7]:
!cython_freeze -o freeze/nCr.c freeze/combinatorics.pyx freeze/lcmath.pyx

2. 转译依赖cython的源码

In [9]:
!cython -o freeze/combinatorics.c freeze/combinatorics.pyx

In [10]:
!cython -o freeze/lcmath.c freeze/lcmath.pyx

3. 编译链接转译好的c文件.需要注意在编译`.o`文件时需要指定python的头文件,在链接各个`.o`文件时需要指定python的动态连接库

In [18]:
!gcc -I /Users/mac/micromamba/envs/py3.10/include/python3.10 -o freeze/nCr.o -c freeze/nCr.c

In [19]:
!gcc -I /Users/mac/micromamba/envs/py3.10/include/python3.10 -o freeze/combinatorics.o -c freeze/combinatorics.c

In [20]:
!gcc -I /Users/mac/micromamba/envs/py3.10/include/python3.10 -o freeze/lcmath.o -c freeze/lcmath.c

In [24]:
!gcc -L/Users/mac/micromamba/envs/py3.10/lib -lpython3.10 freeze/nCr.o freeze/combinatorics.o freeze/lcmath.o -o nCr

4. 测试使用可执行文件.

在使用前我们需要先加载python的动态链接库,当然也可以将python动态链接库的查找地址直接写进环境变量`LD_LIBRARY_PATH`中

In [27]:
!export LD_LIBRARY_PATH="/Users/mac/micromamba/envs/py3.10/lib" && ./nCr

USAGE: ./nCr n r
Prints n-choose-r.


In [28]:
!export LD_LIBRARY_PATH="/Users/mac/micromamba/envs/py3.10/lib" && ./nCr 15812351235 12

5.100280939988478e+113


### 自定义python解释器扩展

如果只是构造可执行文件,cython_freeze可以说大材小用,毕竟带个python解释器基本没什么实用性.cython_freeze实际更多的是用于自定义python解释器扩展.什么是自定义python解释器扩展?通常自定义python解释器扩展就是增加业务专用的模块,全局变量,全局类型,全局函数等,具体什么样可以参考pyspark的交互环境.

我们依然使用上面的例子,通过改变编译过程来将`nCr`函数嵌入自定义python解释器作为全局函数

1. 转译入口c源码,这一步加入选项`--pymain`可以取消将第一个源文件的`__main__`作为入口函数的操作,这样就可以实现这一功能.

In [29]:
!cython_freeze --pymain freeze/combinatorics.pyx freeze/lcmath.pyx -o freeze/mypython.c

In [30]:
!gcc -I /Users/mac/micromamba/envs/py3.10/include/python3.10 -c freeze/mypython.c -o freeze/mypython.o

In [33]:
!gcc -L/Users/mac/micromamba/envs/py3.10/lib -lpython3.10 freeze/mypython.o freeze/combinatorics.o freeze/lcmath.o -o mypython

2. 构造一个`startup.py`文件设置启动项

```python
from combinatorics import nCr
```

3. 设置环境变量`export PYTHONSTARTUP=/Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPythonAsGlueLanguage/附录/Cython使用简介/工具链/startup.py`指定启动加载项

4. 进入自定义python环境`./mypython`,执行`>>> nCr(15812351235, 12)`


## ipython扩展

cython提供了官方的ipython扩展,使用时先试用`%load_ext cython`加载cython扩展,之后就可以使用了.官方扩展有3个

In [34]:
%load_ext cython

+ `%%cython_inline`,inline模式执行cython代码,所谓inline模式类似python中的eval,它可以使用和影响python中的变量,使用`return`就可以返回结果到output中

In [35]:
a = 1
b = 2

In [38]:
%%cython_inline

return a+b

Compiling /Users/mac/Library/Caches/cython/inline/_cython_inline_363a62b67df1fd37db474da6479c306c504718b8.pyx because it changed.
[1/1] Cythonizing /Users/mac/Library/Caches/cython/inline/_cython_inline_363a62b67df1fd37db474da6479c306c504718b8.pyx


3

+ `%%cython_pyximport modulename`,将cell中的代码写入notebook同目录下以`modulename`为名的同名`.pyx`文件中,这样后续就可以使用`pyximport`导入使用了

In [39]:
%%cython_pyximport cf
def f(x):
    return 2.0*x

/Users/mac/micromamba/envs/py3.10/lib/python3.10/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPythonAsGlueLanguage/附录/Cython使用简介/工具链/cf.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [42]:
import pyximport; pyximport.install()
import cf
cf.f(10)

20.0

+ `%%cython`,通用的cython工具,可以使用参数设置编译选项,主要的选项有:
    + `--cplus`,转译为c++而非c
    + `-2`/`-3`,指定使用的python语法版本
    + `-c`,指定传给编译器的选项,比如`-c=-Wno-unreachable-code`
    + `-l`,指定要连接的动态/静态链接库
    + `-L`,指定要连接的动态/静态链接库所在文件夹
    + `-I`,指定要连接的动态/静态链接库头文件所在文件夹
    + `-S`,指定要连接的源文件

需要注意`%%cython`无法获取上下文中python的变量,而且不同块的`%%cython`块之间也无法共享命名空间,因此一般是用来定义函数,然后再在外面调用这些cython函数.

In [46]:
%%cython -3 --cplus

def f2(x):
    return 2.0*x

In [47]:
f2(a)

2.0

In [49]:
%%cython -3 --cplus

f2(a)


Error compiling Cython file:
------------------------------------------------------------
...

f2(a)
^
------------------------------------------------------------

/Users/mac/.cache/ipython/cython/_cython_magic_8b55e1d58827f111c1e0d1c38d8cd196b08a93f3.pyx:2:0: undeclared name not builtin: f2

Error compiling Cython file:
------------------------------------------------------------
...

f2(a)
   ^
------------------------------------------------------------

/Users/mac/.cache/ipython/cython/_cython_magic_8b55e1d58827f111c1e0d1c38d8cd196b08a93f3.pyx:2:3: undeclared name not builtin: a
